# CHECK XSHOOTER TRACE

In [1]:
from __future__ import (print_function, absolute_import)
from __future__ import (division, unicode_literals)

# importing

import numpy as np
from astropy.io import fits
from astropy import units
from glob import glob
import os
import argparse

from pypeit import ginga
from pypeit.core import pixels
from pypeit import traceslits
from pypeit import processimages
from pypeit import scienceimage
from pypeit import arcimage
from pypeit.core import arc
from pypeit import wavecalib
from pypeit import wavetilts
from pypeit import waveimage
from pypeit import flatfield
from pypeit import traceimage
from pypeit import specobjs
from pypeit import utils
from pypeit import biasframe

# Spectrgraph and Settings
from pypeit.spectrographs.util import load_spectrograph
from pypeit.par import pypeitpar
from pypeit import ginga
from pypeit.core.skysub import global_skysub
from pypeit.core import skysub
from pypeit.core import extract
from pypeit.core.flux import generate_sensfunc
#from pypeit.core.flux import generate_sensfunc
from linetools import utils as ltu

from IPython import embed
from matplotlib import pyplot as plt

In [2]:
# import os
# os.system('ginga --modules=RC&')

In [3]:
spectro_name = 'vlt_xshooter_vis'
spectrograph = load_spectrograph(spectrograph=spectro_name)

### Load settings

## Detector settings
par = spectrograph.default_pypeit_par()

## Trace settings
par['calibrations']['traceframe']['process']['overscan'] = 'median'

# Arc settings
par['calibrations']['arcframe']['process']['overscan'] = 'median'

# Flat settings
par['calibrations']['pixelflatframe']['process']['overscan'] = 'median'

# Slits settings
par['calibrations']['slits']['polyorder'] = 6
par['calibrations']['slits']['number'] = -1
par['calibrations']['slits']['maxshift'] = 0.5
par['calibrations']['slits']['sigdetect'] = 8.0
par['calibrations']['slits']['fracignore'] = 0.01
par['calibrations']['slits']['pcatype'] = 'pixel'


par['calibrations']['slits']

Parameter       Value       Default     Type        Callable
------------------------------------------------------------
function        legendre    legendre    str         False   
polyorder       6           3           int         False   
medrep          0           0           int         False   
number          -1          -1          int         False   
trim            3, 3        3, 3        tuple       False   
maxgap          None        None        int         False   
maxshift        0.5         0.15        int, float  False   
pad             0           0           int         False   
sigdetect       8.0         20.0        int, float  False   
fracignore      0.01        0.01        float       False   
min_slit_width  6.0         6.0         float       False   
diffpolyorder   2           2           int         False   
single          []          []          list        False   
sobel_mode      nearest     nearest     str         False   
pcatype         pixel   

In [4]:
spectrograph.detector[0]['datasec'] = '[29:1970,1:]'
spectrograph.detector[0]['oscansec'] = '[2:7,1:]'
print(spectrograph.detector)

[Parameter      Value         Default  Type              Callable
----------------------------------------------------------------
dataext        0             0        int               False   
dispaxis       0             0        int               False   
dispflip       False         False    bool              False   
xgap           0.0           0.0      int, float        False   
ygap           0.0           0.0      int, float        False   
ysize          1.0           1.0      int, float        False   
platescale     0.16          0.135    int, float        False   
darkcurr       0.0           0.0      int, float        False   
saturation     65535.0       65535.0  int, float        False   
mincounts      -1000.0       -1000.0  int, float        False   
nonlinear      0.86          0.86     int, float        False   
numamplifiers  1             1        int               False   
gain           0.595         4.0      int, float, list  False   
ronoise        3.1      

# Testing Binning

In [5]:
xs_dir = '/home/ema/Dropbox/XshooterTestBed/VLT_XSHOOTER/'

# binning = '1x1'
binning = '1x2'
# binning = '2x2'

# Testing 1x1 binning

In [6]:
if binning is '1x1':
    print('1x1 binning')
    xsvis1x1_dir = xs_dir+'VIS_1x1/'

    bias_files = [xsvis1x1_dir+'XSHOO.2010-04-28T10:23:42.901.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T10:26:26.465.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T10:29:10.029.fits.gz']
    flat_files = [xsvis1x1_dir+'XSHOO.2010-04-28T12:24:28.466.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T12:25:39.192.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T12:26:49.168.fits.gz']

# Testing 1x2 binning

In [7]:
if binning is '1x2':
    print('1x2 binning')
    xsvis1x2_dir = xs_dir+'VIS_1x2/'

    bias_files = [xsvis1x2_dir+'XSHOO.2016-08-02T10:45:45.410.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T10:47:16.857.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T10:48:48.184.fits.gz']

    flat_files = [xsvis1x2_dir+'XSHOO.2016-08-02T13:36:24.258.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T13:37:24.132.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T13:38:24.917.fits.gz']

1x2 binning


# Testing 2x2 binning

In [8]:
if binning is '2x2':
    print('2x2 binning')
    xsvis2x2_dir = xs_dir+'VIS_2x2/'

    bias_files = [xsvis2x2_dir+'XSHOO.2016-10-08T10:07:38.147.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T10:08:35.051.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T10:09:30.026.fits.gz']

    flat_files = [xsvis2x2_dir+'XSHOO.2016-10-08T13:03:20.598.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T13:03:57.921.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T13:04:35.024.fits.gz']

In [9]:
par['calibrations']['biasframe']['number'] = 3

bImage = biasframe.BiasFrame(spectrograph,
                             file_list=bias_files,
                             par=par['calibrations']['biasframe'])

bimage = bImage.process()

[WARNING] :: processimages.py 448 process() - Your images have not been bias subtracted!
[INFO]    :: combine.py 62 comb_frames() - Combining 3 bias frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: combine.py 215 comb_frames() - 3 bias frames combined successfully!


In [10]:
tImage = traceimage.TraceImage(spectrograph,
                               file_list=flat_files,
                               par=par['calibrations']['traceframe'])

tflat = tImage.process(bias_subtract=bimage,
                       trim=True)
mstrace = tflat.copy()

[INFO]    :: processimages.py 281 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: processimages.py 288 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 288 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 288 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: combine.py 62 comb_frames() - Combining 3 trace_image frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with

In [11]:
pixlocn = pixels.gen_pixloc(tImage.stack.shape)
bpm = spectrograph.bpm(shape=tflat.shape, det=1)

[INFO]    :: pixels.py 37 gen_pixloc() - Deriving physical pixel locations on the detector
[INFO]    :: pixels.py 40 gen_pixloc() - Pixel gap in the dispersion direction = 0.000
[INFO]    :: pixels.py 41 gen_pixloc() - Pixel size in the dispersion direction = 1.000
[INFO]    :: pixels.py 44 gen_pixloc() - Pixel gap in the spatial direction = 0.000
[INFO]    :: pixels.py 45 gen_pixloc() - Pixel size in the spatial direction = 1.000
[INFO]    :: pixels.py 50 gen_pixloc() - Saving pixel locations


In [12]:
ginga.show_image(bpm)
_, _ = ginga.show_image(tflat*(1.-bpm))

In [13]:
tSlits = traceslits.TraceSlits(mstrace,
                               pixlocn,
                               par=par['calibrations']['slits'],
                               binbpx=bpm)
tslits_dict = tSlits.run(plate_scale=spectrograph.detector[0]['platescale'])

[INFO]    :: trace_slits.py 813 edgearr_from_binarr() - Detecting slit edges in the mstrace image
[INFO]    :: trace_slits.py 874 edgearr_from_binarr() - Applying bad pixel mask
[INFO]    :: trace_slits.py 933 edgearr_add_left_right() - 17 left edges and 18 right edges were found in the trace
[INFO]    :: trace_slits.py 960 edgearr_add_left_right() - Assigning slit edge traces
[INFO]    :: traceslits.py 361 _assign_edges() - Assigning left slit edges
             trace_slits.py 160 assign_slits() - Outer left edge loop, Iteration 1
             trace_slits.py 298 assign_slits() -   Inner loop, Iteration 1, 25 left edges assigned (25 total)
             trace_slits.py 298 assign_slits() -   Inner loop, Iteration 2, 4 left edges assigned (29 total)
             trace_slits.py 298 assign_slits() -   Inner loop, Iteration 3, 2 left edges assigned (31 total)
             trace_slits.py 298 assign_slits() -   Inner loop, Iteration 4, 4 left edges assigned (35 total)
             trace_slits.

In [14]:
# Look at what TraceSlits was actually trying to trace
viewer, ch = ginga.show_image(tSlits.edgearr)
# Look at the sawtooth convolved image
viewer, ch = ginga.show_image(tSlits.siglev)

tmp = tSlits.edgearr * 100.
tmp[np.where(tmp == 0.)] = 1.
ginga.show_image(tSlits.mstrace * tmp)
ginga.show_slits(viewer, ch, tSlits.lcen, tSlits.rcen, slit_ids = np.arange(tSlits.lcen.shape[1]) + 1, pstep=50)